In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [110]:
import os
from pathlib import Path

import paramiko

import getpass
import gc
import time

import natsort

from tqdm import tqdm

In [3]:
%load_ext autoreload
%autoreload 2
import bnpm
from bnpm import server

In [4]:
# paramiko.util.log_to_file("paramiko.log")

----
# ==== Optional ====
### transfer raw data from local machine to `MICROSCOPE`

In [5]:
# ## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to (MICROSCOPE)
# dir_data_local = '/media/rich/bigSSD/tmp_data/'
# dir_data_MICROSCOPE = '/n/files/Neurobio/MICROSCOPE/Rich/data/test'  ## contents of dir_data_local will be dumped into dir_data_MICROSCOPE

# print(f'contents of    {dir_data_local}    will be copied to    {dir_data_MICROSCOPE}')

In [6]:
# remote_host_transfer = "transfer.rc.hms.harvard.edu"
# remote_host_compute = "o2.hms.harvard.edu"
# username = input('Username: ')
# pw = util.pw_encode(getpass.getpass(prompt='Password: '))

## Prepare `sftp` and `ssh` objects

In [7]:
# sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
# sftp.connect(username=username, password=util.pw_decode(pw))

In [8]:
# # sftp.mkdir_safe(dir_data_remote)
# sftp.put_dir(dir_data_local, dir_data_MICROSCOPE)

------------
# ==== Start ====

## Prepare username + password

In [10]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = server.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [11]:
sftp = server.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=server.pw_decode(pw))

In [13]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,    
)

Exception ignored in: <function ssh_interface.__del__ at 0x7fd465ef9430>
Traceback (most recent call last):
  File "/media/rich/Home_Linux_partition/github_repos/basic_neural_processing_modules/bnpm/server.py", line 442, in __del__
    self.ssh.close()
AttributeError: 'NoneType' object has no attribute 'close'


Duo two-factor login for rh183

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4151
 2. Phone call to XXX-XXX-4151
 3. SMS passcodes to XXX-XXX-4151

Passcode or option (1-3): 
1



Pushed a login request to your device...

Success. Logging you in...
Welcome to O2 (Orchestra 2)!

   ___    ____
  / _ \  |___ \
 | | | |   __) |
 | |_| |  / __/
  \___/  |_____|


You've landed on transfer05 which is a
4 core system with 7.62 GiB memory
running kernel 3.10.0 born on 2020-07-28

==== O2 ===================================================================

  News (Sep 8 2022)
  

  +-------------------------------------------------------------------------------+
  | ###################### Try out the new O2 Portal! ########################### |
  |                                                                               |
  | * Access O2 via the web! Just point your web browser to                       |
  |   o2portal.rc.hms.harvard.edu and login with

# Download data to local

In [16]:
sftp_t = server.sftp_interface(ssh_client=ssh_t)

In [96]:
paths_stat_raw = sftp_t.search_recursive(
#     path='/n/files/Neurobio/MICROSCOPE/Rich/data/res2p/old_data/scanimage data/round 5 experiments/mouse 2_6',
    path='/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L',
    search_pattern_re='stat.npy',
    max_depth=10,
);

/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230115/scanimage_data/baseline/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230115/scanimage_data/post/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230115/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230112/scanimage_data/baseline/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230112/scanimage_data/post/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230112/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230113/scanimage_data/baseline/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/

In [98]:
def str_contains(s, t):
    import numpy as np
    return np.where([s_i.find(t) >= 0 for s_i in s])[0][0]

In [105]:
paths_stat = natsort.natsorted([p for p in paths_stat_raw if '/exp/' in p])
display(paths_stat)

dates = natsort.natsorted([Path(p).parent.parent.parent.parent.parent.parent.name for p in paths_stat])
display(dates)

paths_stat = {date: paths_stat_raw[str_contains(paths_stat_raw, date)] for date in dates}
display(paths_stat)

['/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230111/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy',
 '/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230112/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy',
 '/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230113/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy',
 '/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230114/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy',
 '/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230115/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy',
 '/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230116/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy',
 '/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230117/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy',
 '/n/data1/hms/neurobio/sabatini/gyu/analysis/su

['20230111',
 '20230112',
 '20230113',
 '20230114',
 '20230115',
 '20230116',
 '20230117',
 '20230118',
 '20230119']

{'20230111': '/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230111/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy',
 '20230112': '/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230112/scanimage_data/baseline/jobNum_0/suite2p/plane0/stat.npy',
 '20230113': '/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230113/scanimage_data/baseline/jobNum_0/suite2p/plane0/stat.npy',
 '20230114': '/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230114/scanimage_data/baseline/jobNum_0/suite2p/plane0/stat.npy',
 '20230115': '/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230115/scanimage_data/baseline/jobNum_0/suite2p/plane0/stat.npy',
 '20230116': '/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230116/scanimage_data/baseline/jobNum_0/suite2p/plane0/stat.npy',
 '20230117': '/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20

In [106]:
paths_ops = {date: str(Path(path).parent / 'ops.npy') for date, path in paths_stat.items()}

In [107]:
assert all([sftp_t.exists(path) for path in paths_stat.values()])
assert all([sftp_t.exists(path) for path in paths_ops.values()])

In [ ]:
dir_save_local = r'/media/rich/bigSSD/analysis_data/statFiles_mouse_1511L'

In [108]:
[(Path(str(Path(dir_save_local) / date)).mkdir(parents=True, exist_ok=True), sftp_t.sftp.get(
    remotepath=path,
    localpath=str(Path(dir_save_local) / date / Path(path).name),
)) for date,path in tqdm(paths_stat.items())];

In [109]:
[(Path(str(Path(dir_save_local) / date)).mkdir(parents=True, exist_ok=True), sftp_t.sftp.get(
    remotepath=path,
    localpath=str(Path(dir_save_local) / date / Path(path).name),
)) for date,path in tqdm(paths_ops.items())];

# Run remotely

In [21]:
## initialize ssh_compute
ssh_c = util.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,    
)

Duo two-factor login for rh183

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4151
 2. Phone call to XXX-XXX-4151
 3. SMS passcodes to XXX-XXX-4151

Passcode or option (1-3): 
1



Pushed a login request to your device...

Success. Logging you in...
Welcome to O2 (Orchestra 2)!

   ___    ____
  / _ \  |___ \
 | | | |   __) |
 | |_| |  / __/
  \___/  |_____|


You've landed on login06 which is a
8 core system with 15.49 GiB memory
running kernel 3.10.0 born on 2020-07-07

==== O2 ===================================================================

  News (Sep 8 2022)
  

  +-------------------------------------------------------------------------------+
  | ###################### Try out the new O2 Portal! ########################### |
  |                                                                               |
  | * Access O2 via the web! Just point your web browser to                       |
  |   o2portal.rc.hms.harvard.edu and login with y

## Pull/update repo

In [70]:
ssh_c.send('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/ROICaT')
ssh_c.send_receive('git pull')

ssh_c.expect(str_success=f'[{username}', total_timeout=2);

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (3/3), done.

remote: Total 9 (delta 5), reused 9 (delta 5), pack-reused 0

Unpacking objects:  44% (4/9)   
Unpacking objects: 100% (9/9), done.



## Activate environment

In [52]:
ssh_c.send_receive('source activate ROICaT')

ssh_c.expect(str_success=f'[{username}', total_timeout=2);

(ROICaT) [rh183@login06 ROICaT]$ 


## Prepare directories

In [53]:
import numpy as np

dir_outer = 'round_5_experiments'
mouse_name = 'mouse_2_6'
plane_name = 'exp/suite2p/plane0'
# date = ''

name_slurm = mouse_name + '_' + plane_name[-1:]


dir_data_remote = (Path('/n/data1/hms/neurobio/sabatini/rich/data/res2p/') / dir_outer / mouse_name).as_posix()

dir_saveOutputs = (Path('/n/data1/hms/neurobio/sabatini/rich/analysis/ROI_tracking') / mouse_name).as_posix()

dir_ROInet_networkFiles = Path('/n/data1/hms/neurobio/sabatini/rich/analysis/ROI_tracking/ROInet_temp_networkFiles').as_posix()


name_job = 'jobNum_'


path_dispatcher_local = str(Path(r'/media/rich/Home_Linux_partition/github_repos/ROICaT/scripts/slurm_dispatching/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
path_dispatcher_remote = str((Path(dir_saveOutputs) / 'dispatcher.py'))  ## path to where you want the dispatcher.py file to be copied onto the server
path_script_remote = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/ROICaT/scripts/slurm_dispatching/tracking_script.py'))  ## path to a copy of the remote script.py

In [54]:
display(f'{dir_data_remote = }')
display(f'{dir_saveOutputs = }')
display(f'{dir_ROInet_networkFiles = }')
display(f'{path_dispatcher_local = }')
display(f'{path_dispatcher_remote = }')
display(f'{path_script_remote = }')

"dir_data_remote = '/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_5_experiments/mouse_2_6'"

"dir_saveOutputs = '/n/data1/hms/neurobio/sabatini/rich/analysis/ROI_tracking/mouse_2_6'"

"dir_ROInet_networkFiles = '/n/data1/hms/neurobio/sabatini/rich/analysis/ROI_tracking/ROInet_temp_networkFiles'"

"path_dispatcher_local = '/media/rich/Home_Linux_partition/github_repos/ROICaT/scripts/slurm_dispatching/dispatcher.py'"

"path_dispatcher_remote = '/n/data1/hms/neurobio/sabatini/rich/analysis/ROI_tracking/mouse_2_6/dispatcher.py'"

"path_script_remote = '/n/data1/hms/neurobio/sabatini/rich/github_repos/ROICaT/scripts/slurm_dispatching/tracking_script.py'"

## Prepare commands

In [55]:
## Expectation for dispatch args: 
# path_selfScript = args[0] = path_dispatcher_remote
# dir_save = args[1] = dir_S2pOutput_remote
# path_script = args[2] = path_s2pScript_remote
# name_job = args[3] = name_job
# dir_fastDisk = args[4] = dir_fastDisk_remote
# name_slurm = args[5] = name_slurm
# dir_data = args[6] = dir_data_remote


prompt_snip = f'[{username}'

commands = {
    'make_dir': f"mkdir -p {dir_data_remote}",
#     'copy_s2p': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
    'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_data_remote} {plane_name} {dir_ROInet_networkFiles}"  ## dispatcher expecting these args as inputs
}
display(commands)

{'make_dir': 'mkdir -p /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_5_experiments/mouse_2_6',
 'dispatch': 'python /n/data1/hms/neurobio/sabatini/rich/analysis/ROI_tracking/mouse_2_6/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/analysis/ROI_tracking/mouse_2_6 /n/data1/hms/neurobio/sabatini/rich/github_repos/ROICaT/scripts/slurm_dispatching/tracking_script.py jobNum_ mouse_2_6_0 /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_5_experiments/mouse_2_6 exp/suite2p/plane0 /n/data1/hms/neurobio/sabatini/rich/analysis/ROI_tracking/ROInet_temp_networkFiles'}

## Upload `dispatcher.py` file

In [71]:
sftp.mkdir_p(Path(path_dispatcher_remote).parent.as_posix())
sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));

## Dispatch `remote_run_s2p.py`

In [79]:
ssh_c.send(commands['dispatch']);

ssh_c.expect(str_success=prompt_snip);

workFiles/data1/hms/neurobio/sabatini/rich/analysis/ROI_tracking/ROInet_temp_net 

/n/data1/hms/neurobio/sabatini/rich/analysis/ROI_tracking/mouse_2_6/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/analysis/ROI_tracking/mouse_2_6 /n/data1/hms/neurobio/sabatini/rich/github_repos/ROICaT/scripts/slurm_dispatching/tracking_script.py jobNum_ /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_5_experiments/mouse_2_6

Submitting job: jobNum_ 0

sbatch: Setting QOS to gpuquad_qos
Submitted batch job 1584725

(ROICaT) [rh183@login06 ROICaT]$ 


### check on job

In [87]:
ssh_c.send(cmd=f'squeue -u {username} --Format=jobid,name,partition,state,timeused,starttime,timelimit,reasonlist,minmemory,mincpus,gres');
test = ssh_c.expect(str_success=f"[{username}", total_timeout=10);

tate,timeused,starttime,timelimit,reasonlist,minmemory,mincpus,gres
JOBID               NAME                PARTITION           STATE               TIME                START_TIME          TIME_LIMIT          NODELIST(REASON)    MIN_MEMORY          MIN_CPUS            TRES_PER_NODE       
1584725             mouse_2_6_0         gpu_requeue         PENDING             0:00                N/A                 1:00:00             (None)              64G                 8                   gres:gpu:rtx6000:1  
1584500             bash                interactive         RUNNING             4:45                2023-01-18T18:46:14 30:00               compute-a-16-165    8G                  1                   N/A                 
(ROICaT) [rh183@login06 ROICaT]$ 


In [686]:
## I think this code block caused an out of memory error on a few of my jobs... not sure why

# ssh_c.send(cmd=f"cat {(Path(dir_S2pOutput_remote) / (name_job+'0') / 'print*').as_posix()}")
# ssh_c.expect(str_success=f"[{username}", total_timeout=1);

In [687]:
ssh_c.send(cmd=f"cat {(Path(dir_saveOutputs) / (name_job+'0') / 'print*').as_posix()}")
ssh_c.expect(str_success=f"RUN COMPLETE", recv_timeout=0.3, total_timeout=60*60*10, sleep_time=0.1, verbose=False);

print(f'RUN COMPLETE!!!     {time.ctime()}')

KeyboardInterrupt: 

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs back from `data1` to `MICROSCOPE`

## Prepare username + password

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `ssh` object

In [ ]:
## initialize ssh_transfer
ssh_t = util.ssh_interface(
    nbytes_toReceive=4096,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,
)

In [274]:
dir_S2pOutput_remote = dir_S2pOutput_remote  ## from above
dir_s2pOutput_MICROSCOPE = (Path(dir_data_MICROSCOPE) / 'suite2p_o2_output').as_posix()  ## from above
print(f'contents of    {dir_S2pOutput_remote}    will be copied to    {dir_s2pOutput_MICROSCOPE}')

contents of    /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG21/2022_05_14    will be copied to    /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output


In [275]:
commands = {
    'make_dir': f"mkdir -p {dir_s2pOutput_MICROSCOPE}",
    'copy_s2p': f"cp -r {(Path(dir_S2pOutput_remote)).as_posix()}/. {dir_s2pOutput_MICROSCOPE}",
}
display(commands)

{'make_dir': 'mkdir -p /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output',
 'copy_s2p': 'cp -r /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG21/2022_05_14/. /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output'}

In [276]:
ssh_t.send_receive(commands['make_dir']);

mkdir -p /n/files/Neurobio/MICROSCOPE/Ally/Mesoscop 
e/AEG21/2022_05_14/suite2p_o2_output



In [277]:
ssh_t.send(commands['copy_s2p'])
ssh_t.expect(str_success=f'[{username}', total_timeout=120, verbose=True);

(base) [rh183@transfer03 ~]$ 


## Clean up

In [ ]:
## delete password
del pw
gc.collect()
    
## close ssh
ssh_t.close()
del ssh_t
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs from `MICROSCOPE` to local machine

In [39]:
dir_s2pOutput_MICROSCOPE = dir_s2pOutput_MICROSCOPE  ## from above
dir_s2pOutput_local = str(Path(dir_data_local).resolve() / 'suite2p_o2_output')
print(f'contents of    {dir_s2pOutput_MICROSCOPE}    will be copied to    {dir_s2pOutput_local}')

contents of    /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output    will be copied to    /media/rich/bigSSD/tmp_data/suite2p_o2_output


In [57]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [40]:
sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=util.pw_decode(pw))

In [41]:
sftp.get_dir(dir_s2pOutput_MICROSCOPE, dir_s2pOutput_local)

downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/params.json   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/params.json
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/print_log_53419173.log   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/print_log_53419173.log
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/batch_run_output.png   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/batch_run_output.png
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/log.txt   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/log.txt
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/sbatch_config.sh   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/sbatch_config.sh
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/remote_run_s2p.py   to   /medi

## Clean up

In [43]:
## delete password
del pw
gc.collect()

## close sftp
sftp.close()
del sftp

In [42]:
ssh_t.close()
ssh_c.close()